# Temporal analysis of messages by channel

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import dataset
df = pd.read_json("../../Datasets/voluntariosdanavalencia_old.json") # Import data from Voluntarios de Valencia groupchat.

### Check channels

In [3]:
# Check data structure
print(df.shape)
print(df.columns)
df.head()

(18064, 17)
Index(['id', 'date', 'message', 'reply_to_msg_id', 'views', 'forwards',
       'edit_date', 'pinned', 'post_author', 'sender_id', 'sender_username',
       'sender_name', 'has_media', 'reactions', 'subgroup', 'topic_name',
       'reply'],
      dtype='object')


,id,date,message,reply_to_msg_id,views,forwards,edit_date,pinned,post_author,sender_id,sender_username,sender_name,has_media,reactions,subgroup,topic_name,reply
0,3,2024-10-30 12:39:24+00:00,Os paso por aquí el correo,NaN,NaN,NaN,None,False,NaN,1110241832,Manuel_deRonde,Manuel DeRonde - Coordinador Civil 🫱🏻‍🫲🏼,False,[],None,Discusión general,False
1,4,2024-10-30 12:40:50+00:00,LO AVISÉ EL VIERNES PASADO Y NADIE ME HIZO CAS...,NaN,46823.0,1057.0,2024-11-01T18:21:38+00:00,False,NaN,1110241832,Manuel_deRonde,Manuel DeRonde - Coordinador Civil 🫱🏻‍🫲🏼,True,"[{'user_id': 5887977931, 'username': '', 'disp...",None,Discusión general,False
2,5,2024-10-30 12:43:17+00:00,,NaN,NaN,NaN,2024-11-03T22:44:05+00:00,False,NaN,1110241832,Manuel_deRonde,Manuel DeRonde - Coordinador Civil 🫱🏻‍🫲🏼,True,"[{'user_id': 6970134793, 'username': '', 'disp...",None,Discusión general,False
3,6,2024-10-30 12:43:23+00:00,Probad si podeis,NaN,NaN,NaN,None,False,NaN,1110241832,Manuel_deRonde,Manuel DeRonde - Coordinador Civil 🫱🏻‍🫲🏼,False,[],None,Discusión general,False
4,7,2024-10-30 12:46:02+00:00,Durante la tormenta al parecer sí estaba opera...,4.0,NaN,NaN,None,False,NaN,941865373,Camelocota,Víctor Conejero,False,[],Thread_4,Discusión general,True


In [7]:
# Convert date to datetime and sort by time
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f"Total messages: {len(df)}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

Total messages: 18064
Date range: 2024-10-30 12:39:24+00:00 to 2025-11-10 11:06:47+00:00


In [10]:
# Calculate waiting times (inter-arrival times) between consecutive messages
import numpy as np

# Calculate time differences in seconds
df['waiting_time'] = df['date'].diff().dt.total_seconds()

# Remove the first NaN value and negative values (if any)
waiting_times = df['waiting_time'].dropna()
waiting_times = waiting_times[waiting_times > 0]

In [8]:
# Check possible values for 'topic_name' column
print(df['topic_name'].unique())

['Discusión general' 'Bienvenid@s a la Red de Voluntari@s (leed primero)'
 'Transporte de personas a Comunidad Valenciana'
 'Grupos de Acción Voluntarios en Terreno' 'Asistencia Jurídica/Seguros'
 'Asistencia Técnica y Maquinaria (mecánica, fontanería, electricidad, agricola)'
 'Asistencia animales compañia y otros'
 'Asistencia Informática/Soporte EAT'
 'Logística Interna Valencia(puntos de recepción)' 'Ayuda de Empresas'
 'Investigación Ciudadana: Responsabilidades Políticas'
 'Asistencia a Voluntarios (Comida y Acogida))'
 'Asis. Grupos Vulnerables (Maternidad , Infancia, Mayores, etc)'
 'Material Video/Fotográfico' 'Webs Esenciales'
 'Mensajes de Ayuda (Ofrezco/Necesito)'
 'Voluntarios de fuera de Valencia (Madrid, Cataluña, Alicante, Castellón)'
 'Viviendas Edificación , Muebles y Acondicionamiento']


In [9]:
# Generate dataframe for each channel
df_general = df[df['topic_name'] == 'Discusión general']
df_bienvenidos = df[df['topic_name'] == 'Bienvenid@s a la Red de Voluntari@s (leed primero)']
df_tranporte = df[df['topic_name'] == 'Transporte de personas a Comunidad Valenciana']
df_grupos_accion = df[df['topic_name'] == 'Grupos de Acción Voluntarios en Terreno']
df_juridica = df[df['topic_name'] == 'Asistencia Jurídica/Seguros']
df_tecnica = df[df['topic_name'] == 'Asistencia Técnica y Maquinaria (mecánica, fontanería, electricidad, agricola)']
df_animales = df[df['topic_name'] == 'Asistencia animales compañia y otros']
df_informatica = df[df['topic_name'] == 'Asistencia Informática/Soporte EAT']
df_logistica = df[df['topic_name'] == 'Logística Interna Valencia(puntos de recepción)']
df_ayudaempresas = df[df['topic_name'] == 'Ayuda de Empresas']
df_investigacion = df[df['topic_name'] == 'Investigación Ciudadana: Responsabilidades Políticas']
df_asistencia_voluntarios = df[df['topic_name'] == 'Asistencia a Voluntarios (Comida y Acogida))']
df_asistencia_vulnerables = df[df['topic_name'] == 'Asis. Grupos Vulnerables (Maternidad , Infancia, Mayores, etc)']
df_material = df[df['topic_name'] == 'Material Video/Fotográfico']
df_webs = df[df['topic_name'] == 'Webs Esenciales']
df_mensajes_ayuda = df[df['topic_name'] == 'Mensajes de Ayuda (Ofrezco/Necesito)']
df_voluntarios_fuera = df[df['topic_name'] == 'Voluntarios de fuera de Valencia (Madrid, Cataluña, Alicante, Castellón)']
df_viviendas = df[df['topic_name'] == 'Viviendas Edificación , Muebles y Acondicionamiento']


In [11]:
# Check lenght for the different channels
print(f"Total messages in 'Discusión general': {len(df_general)}")
print(f"Total messages in 'Bienvenid@s a la Red de Voluntari@s (leed primero)': {len(df_bienvenidos)}")
print(f"Total messages in 'Transporte de personas a Comunidad Valenciana': {len(df_tranporte)}")
print(f"Total messages in 'Grupos de Acción Voluntarios en Terreno': {len(df_grupos_accion)}")
print(f"Total messages in 'Asistencia Jurídica/Seguros': {len(df_juridica)}")
print(f"Total messages in 'Asistencia Técnica y Maquinaria (mecánica, fontanería, electricidad, agricola)': {len(df_tecnica)}")
print(f"Total messages in 'Asistencia animales compañia y otros': {len(df_animales)}")
print(f"Total messages in 'Asistencia Informática/Soporte EAT': {len(df_informatica)}")
print(f"Total messages in 'Logística Interna Valencia(puntos de recepción)': {len(df_logistica)}")
print(f"Total messages in 'Ayuda de Empresas': {len(df_ayudaempresas)}")
print(f"Total messages in 'Investigación Ciudadana: Responsabilidades Políticas': {len(df_investigacion)}")
print(f"Total messages in 'Asistencia a Voluntarios (Comida y Acogida))': {len(df_asistencia_voluntarios)}")
print(f"Total messages in 'Asis. Grupos Vulnerables (Maternidad , Infancia, Mayores, etc)': {len(df_asistencia_vulnerables)}")
print(f"Total messages in 'Material Video/Fotográfico': {len(df_material)}")
print(f"Total messages in 'Webs Esenciales': {len(df_webs)}")
print(f"Total messages in 'Mensajes de Ayuda (Ofrezco/Necesito)': {len(df_mensajes_ayuda)}")
print(f"Total messages in 'Voluntarios de fuera de Valencia (Madrid, Cataluña, Alicante, Castellón)': {len(df_voluntarios_fuera)}") 
print(f"Total messages in 'Viviendas Edificación , Muebles y Acondicionamiento': {len(df_viviendas)}")

Total messages in 'Discusión general': 256
Total messages in 'Bienvenid@s a la Red de Voluntari@s (leed primero)': 2856
Total messages in 'Transporte de personas a Comunidad Valenciana': 2076
Total messages in 'Grupos de Acción Voluntarios en Terreno': 1052
Total messages in 'Asistencia Jurídica/Seguros': 819
Total messages in 'Asistencia Técnica y Maquinaria (mecánica, fontanería, electricidad, agricola)': 488
Total messages in 'Asistencia animales compañia y otros': 1033
Total messages in 'Asistencia Informática/Soporte EAT': 1964
Total messages in 'Logística Interna Valencia(puntos de recepción)': 1307
Total messages in 'Ayuda de Empresas': 300
Total messages in 'Investigación Ciudadana: Responsabilidades Políticas': 2770
Total messages in 'Asistencia a Voluntarios (Comida y Acogida))': 1163
Total messages in 'Asis. Grupos Vulnerables (Maternidad , Infancia, Mayores, etc)': 144
Total messages in 'Material Video/Fotográfico': 1170
Total messages in 'Webs Esenciales': 12
Total message